In [34]:
import time
import shutil
import math
import re
import csv
import numpy as np
import matplotlib.pyplot as plt

import networkx as nx
from graphviz import Digraph

from IPython.display import display, JSON
from scipy import stats

%matplotlib inline
my_client = fbl.get_client()

[matplotlib.pyplot 2024-11-22 17:22:34,040] Loaded backend module://matplotlib_inline.backend_inline version unknown.


# Graphing FB1G and FB4N for ICONIP24

In [37]:
data = my_client.executeNLPquery('clear')

[FBL NLP 2024-11-22 18:00:48,760] NLP successfully parsed query.


## Start Here: NOI to Investigate

In [425]:
data = my_client.executeNLPquery("add $FB1G$")

[FBL NLP 2024-07-26 22:46:39,600] NLP successfully parsed query.


In [40]:
noi = "FB1G"
noi2 = f"/r(.*){noi}(.*)_R(.*)/r"
prePost = 'post'
nSys = 35
nFilter = '/r(.*)(.*)/r'
nplQuery = f"show {noi2} neurons"
print(nplQuery)
data = my_client.executeNLPquery(nplQuery)

show /r(.*)FB1G(.*)_R(.*)/r neurons
[FBL NLP 2024-11-22 18:03:23,791] NLP successfully parsed query.


In [41]:
data = my_client.executeNLPquery(f"add {prePost}synaptic {nFilter} neurons with at least {nSys} synapses")

[FBL NLP 2024-11-22 18:03:28,922] NLP successfully parsed query.


In [42]:
# timer
start_time = time.time()

neurons = my_client.NLP_result.neurons

hashids = list(neurons.keys())
names = [neurons[k]['uname'] for k in neurons]

preSynapses = []
postSynapses = []

for i, hashid in enumerate(hashids):
    res = my_client.getInfo(hashid)
    data = res["data"]

    if "connectivity" in data:
        presyn = data["connectivity"]["pre"]["details"]
        postsyn = data["connectivity"]["post"]["details"]
        
        for syn in presyn:
            if 'segment' not in syn["uname"]:
                preSynapses.append(["pre", syn["syn_uname"], names[i], syn["uname"], syn["s_rid"], syn["number"], True])

        for syn in postsyn:
            if 'segment' not in syn["uname"]:
                postSynapses.append(["post", syn["syn_uname"], names[i], syn["uname"], syn["s_rid"], syn["number"], True])

connectivity = {"hashids": hashids, "names": names, "preSynapses": preSynapses, "postSynapses": postSynapses}

postSynNumberDict = {}
preSynNumberDict = {}

def process_synapses(synapses, synNumberDict, synType):
    for neuronName in connectivity['names']:
        singleNeuron_synNumber = []
        singleNeuron_synData = []
        for syn in synapses:
            if neuronName == syn[2]:
                singleNeuron_synNumber.append(syn[5])
                singleNeuron_synData.append([syn[3], syn[4], syn[5]])

        synZscore = stats.zscore(singleNeuron_synNumber)
        isnan = np.isnan(synZscore)

        if not np.any(isnan):
            synZmax = max(synZscore)
            print(f'\033[92m{neuronName}: {synType}SynZmax= {synZmax}\033[0m')
        else:
            for syn in synapses:
                if neuronName == syn[2]:
                    syn[6] = False
            print(f'\033[93m{neuronName} Error: Nan Value Found in array\033[0m')
            continue

        for r in range(len(singleNeuron_synData)):
            syn_uname = singleNeuron_synData[r][0]
            if 'segment' not in syn_uname:
                syn_rid = singleNeuron_synData[r][1]
                normSynZ = 1 - ((synZmax - synZscore[r]) / synZmax)
                synNumberDict[syn_rid] = [syn_uname, synZscore[r], normSynZ]

process_synapses(postSynapses, postSynNumberDict, "post")
process_synapses(preSynapses, preSynNumberDict, "pre")

numberOfNodes = len(names)

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
avSynCalc_time = elapsed_time / (len(preSynapses) + len(postSynapses))
avNeuronCalc_time = elapsed_time / numberOfNodes

print ("No. of HashIDs= " + str( len(hashids)))
print ("No. of names= " + str( numberOfNodes ))
print ("No. of preSynapses= " + str( len(preSynapses)))
print ("No. of postSynapses= " + str( len(postSynapses)))
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Avg. time per synapse: {avSynCalc_time:.4f} seconds")
print(f"Avg. time per neuron: {avNeuronCalc_time:.4f} seconds")

FB1G_R_1: postSynZmax= 6.385239743772796
PFR_b(PB11b)_R1_C8_1: postSynZmax= 6.490583708474763
FB3C_R_4: postSynZmax= 11.773209919788671
FC1F(FQ6)_C3_L_1: postSynZmax= 4.86557456639234
FB4C(NO2)_R_1: postSynZmax= 5.131642852656541
FB3E_R_1: postSynZmax= 8.383500744747218
hDeltaC_03_C3_2: postSynZmax= 6.383087576984115
FB3E_L_1: postSynZmax= 9.157765310353927
FB3C_L_3: postSynZmax= 12.44054185943296
FB3C_R_2: postSynZmax= 8.978147818605619
FB4N_R_1: postSynZmax= 6.049566787984049
FB3C_L_4: postSynZmax= 10.337133506116361
FB4N_L_1: postSynZmax= 5.670596292773406
FB3C_L_2: postSynZmax= 9.702704270053886
FB3C_L_1: postSynZmax= 10.758457185515047
FB4C(NO2)_L_1: postSynZmax= 4.596890449389471
FB1G_R_1: preSynZmax= 10.156286862133477
PFR_b(PB11b)_R1_C8_1: preSynZmax= 8.988537410075862
FB3C_R_4: preSynZmax= 12.072559895607208
FC1F(FQ6)_C3_L_1: preSynZmax= 4.733222648109842
FB4C(NO2)_R_1: preSynZmax= 6.090293119420924
FB3E_R_1: preSynZmax= 10.617371489905036
hDeltaC_03_C3_2: preSynZmax= 7.048364

In [43]:
#output to CSV if needed
with open('preOutput.csv','w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['type','syn_name','uname','s_rid','hashid','number'])
    writer.writerows(preSynapses)


In [44]:
with open('postOutput.csv','w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['type','syn_name','uname','s_rid','hashid','number'])
    writer.writerows(postSynapses)

In [45]:
res = my_client.getConnectivity()
nodes = dict(res.graph.nodes(data=True))
edges = list(res.graph.edges(data=True))

#sample outputs 
#print(edges)
#print(edges[0][0])
#display(JSON(nodes))
#print(edges)


In [46]:
display(JSON(nodes))
display(JSON(edges))

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

In [47]:
#G = NetworkX Directional Graph -DiGraph- Creation

G = nx.DiGraph()

for e_pre in nodes:
    try:
        G.add_node(e_pre, uname = nodes[e_pre]['uname'],)
    except KeyError as e:
        print(f"key {e.args[0]} not found")
        continue
    
for edge in edges:
    try:
        G.add_edge(edge[0],edge[1])
    except KeyError as e:
        print(f"key {e.args[0]} error occured")
        continue    

In [48]:
#Calculate Centrality
#add centrality to DiGraph

#Orignally Degree centrality was used
#But has been changed so that key bridge nodes can be identified
#A key bridge may have a at least one weak pre or post synaptic connection
#But a cnetral bridge with weak connectivty on both ends must be avoided
#centralityDict = nx.degree_centrality(G)
centralityDict = nx.betweenness_centrality(G, k=numberOfNodes, normalized=True, weight=None, endpoints=False, seed=None)

for e_pre in nodes:
    G.add_node(e_pre, centrality = centralityDict[e_pre])


In [49]:
#GraphViz
#Rndering the final output ot NeuGFX

#false = line , ortho is curved for example
gvSplines = "false"

graph_struct = {'verbose':'false',
                'splines': gvSplines,
                'pad': '0.5',
                'ranksep': '1.5',
                'concentrate': 'true',
                'newrank': 'true',
                'rankdir': 'LR'}


edgeFontSize = '8.0'
edgeFontColor = "#40e0d0"
edgeFontName = "Courier"
gvSize = '50,50'
#gvLayout = 'dot'
#gvLayout = 'sfdp'
gvLayout = 'sfdp'

g = Digraph('G', 
            filename ='G_ex.gv',
            graph_attr = {  'size': gvSize,
                            'layout':gvLayout,
                            'fontcolor':edgeFontColor,
                            'fontsize':edgeFontSize,
                            'fontname':edgeFontName,
                            'overlap_scaling':'2',
                            'rotation':'0',
                            'mode':'spring',
                            'beautify':'true',
                            'splines': 'curved',
                            'arrowsize': '0.5',
                            'overlap': 'false',
                            'pad': '0.1',
                            'quadtree':'true',
                            'nodesep': '0.3',
                            'outputorder': 'edgesfirst',
                            'ranksep': '0.2',
                            'bgcolor': '#212529',
                            'colorscheme':'rdylgn8',
                            'concentrate': 'true',
                            'newrank': 'true',
                            'rankdir': 'LR'})



In [50]:
def calcColor(normalizedScore):

    pw = pw_default = 0.1
    pc = pc_defult = 'gray'
    retlist = []

    if normalizedScore < 0 :
        pw = pw_default
        pc=  pc_defult
    elif 0 <= normalizedScore < 0.1 :
        pw = pw_default
        pc = 'red'
    elif 0.1 <= normalizedScore < 0.2:
        pw = normalizedScore
        pc = 'orangered3'
    elif 0.2 <= normalizedScore < 0.3:
        pw =  normalizedScore
        pc = 'orangered2'
    elif 0.3 <= normalizedScore < 0.4:
        pw =  normalizedScore
        pc = 'gold' 
    elif 0.4 <= normalizedScore < 0.5:
        pw =  normalizedScore
        pc = 'chartreuse'
    elif 0.5 <= normalizedScore < 0.6:
        pw =  normalizedScore
        pc = 'limegreen' 
    elif 0.6 <= normalizedScore < 0.7:
        pw =  normalizedScore
        pc = 'lime'
    elif 0.7 <= normalizedScore < 0.8:
        pw =  normalizedScore
        pc = 'cyan3' 
    else:
        pw =  normalizedScore
        pc = 'blue' 

    retlist.extend([pw,pc])
    return retlist
 

In [51]:
#EdgeLayout
#Using Centrality
#node_centrality Dict Created

valid_nodes = []
node_centrality = {}
nodeDegreeDict = {}
nodeNameZscoreDict = {}
zScoreMultiplyer = 5
widthColor = []

minNormZscore = 0.15
#display(JSON( G.edges(data=True)))
#print(G.edges(data=True))
for head, tail, data in G.edges(data=True):
    #print('pre= ' + str(head) + 'post= ' + str(tail))
    pre_name = G.nodes(data=True)[head]['uname']
    post_name = G.nodes(data=True)[tail]['uname']
    pw = pw_default = 0.1
    pc = 'grey'

    #set Edge Penwidth and color based on Normalized Z score
    
    #This checks if the head of the edge is presynaptic
    if head in preSynNumberDict.keys():
        widthColor = calcColor(preSynNumberDict[head][2])
        #print ('Head hashid '+ str(head) + '  name= ' + str(pre_name) + ' with val =' + str(preSynNumberDict[head][2])  )
        pw = widthColor[0] * zScoreMultiplyer
        pc = widthColor[1]
        includeEdge = True
        if preSynNumberDict[head][2] <= minNormZscore:
            includeEdge = False

        node_centrality[pre_name] = G.nodes(data=True)[head]['centrality']
        node_centrality[post_name] = G.nodes(data=True)[tail]['centrality']
        nodeNameZscoreDict[pre_name] = pw

    #This checks if the tail of the edge is postsynaptic
    elif tail in postSynNumberDict.keys():
        #print ('Tail hashid '+ str(tail) + 'name = ' + str(post_name) + 'with val= '+ str(postSynNumberDict[tail][2])  )
        widthColor = calcColor(postSynNumberDict[tail][2])
        #print ('Post hashid ' + str(post) + 'with val= '+ str(postSynNumberDict[post][2]) + ' width and color =' + str(widthColor) )
        pw = widthColor[0] * zScoreMultiplyer
        pc = widthColor[1]
        includeEdge = True
        if preSynNumberDict[tail][2] <= minNormZscore:
            
            includeEdge = False
            #print(preSynNumberDict[tail], 'excluded with normalized z score', preSynNumberDict[tail][2])
        
        node_centrality[pre_name] = G.nodes(data=True)[head]['centrality']
        node_centrality[post_name] = G.nodes(data=True)[tail]['centrality']
        nodeNameZscoreDict[pre_name] = pw
    
    
    else:
        print('Both ' + str(head) + 'and' + str(tail) + 'is not synapses!!!')
            

            
    #Don't include edges that are trivial
    if includeEdge:
        g.edge(str(pre_name), str(post_name), 
            labelfloat = 'true', 
            label = 'Zscore = ' + str(round(widthColor[0],2)),
            fontsize = edgeFontSize,
            fontcolor = edgeFontColor,
            fontname = edgeFontName,
            penwidth= str(pw),
            weight='3',
            arrowsize='0.4', 
            color=pc)
        valid_nodes.append(str(pre_name))
        valid_nodes.append(str(post_name))
        valid_nodes = list(set(valid_nodes))

        

In [52]:
#GraphViz node setup
#New version uses centrality



#NODE Attributtes

for _head in valid_nodes:
    #print(_head)
    #If  this is a pre or post synapse
    if '--' in str(_head):
        ht_float = math.floor(node_centrality.get(str(_head))*1000)/1000
        ht       = ht_float if ht_float > 0.05 else 0.05
        g.node(str(_head),
            shape='circle',
            height= str(ht),
            label='',
            fontsize='4.0',
            fixedsize='true',
            color= 'cyan' if ht_float > 0.05 else 'grey',
            style='filled')
    #else must a neuron node 
    else:
        if _head in nodeNameZscoreDict.keys():
            ht_float = math.floor(node_centrality.get(str(_head))*1000)/1000
            calculatedFontSize = str(ht_float * 10)
            centralityLabel = "cent= " + str(ht_float)
            #ht_float += (0.1 * nodeNameZscoreDict[_head])
            g.node(str(_head),
                shape='circle',
                height= str(ht_float),
                fontsize=calculatedFontSize,
                label= str(_head +"\n" + centralityLabel),
                fixedsize='true',
                fontcolor='white',
                color='brown2',
                style='filled')
        else:
            #because of NaN error, we remove this problematic node
            print(str(_head) + ' Not found in nodeNameZscoreDict')
            g.node(str(_head),
                shape='circle',
                height= '0.2',
                fontsize=calculatedFontSize,
                fixedsize='true',
                fontcolor='red',
                color='yellow',
                style='filled')
            
            
        
      


In [53]:


g.attr(size='50,50')
g.verbose = True
try:
    g.save('G_auto.gv')
    g.render('G_auto', format = 'svg', view=False)
    
    
    #g.render('G_auto', format = 'png', view=False)
except Exception as e:
    self.raise_error(e, 'There was an error during diagram generation. Please execute "conda install -c anaconda graphviz" in your terminal in your conda environment, or try to install GraphViz globally from https://graphviz.org/download/.')


#Make a filter str to use in the save file name
nFilterStr = nFilter.replace('$','')

characters_to_replace = ['/r', ']', '[', '.', '*', '(', ')']

# Replace each character in the list
for char in characters_to_replace:
    nFilterStr = nFilterStr.replace(char, '')


noiFileName = str(noi +'_'+ nFilterStr + '_' + str(nSys) + prePost + '_001')
shutil.copy('G_auto.svg', noiFileName + '.svg')

[graphviz.saving 2024-11-22 18:12:19,921] write lines to 'G_auto.gv'
[graphviz.saving 2024-11-22 18:12:19,940] write lines to 'G_auto'
[graphviz.backend.execute 2024-11-22 18:12:19,956] run [PosixPath('dot'), '-Kdot', '-Tsvg', '-O', 'G_auto']


'FB1G__35post_001.svg'

In [54]:
with open('G_auto.svg', 'r') as file:
    data = file.read()

In [55]:
# Update widgets and widget managers throughout the frontend (included for compatibility)
for i in fbl.widget_manager.widgets:
    if fbl.widget_manager.widgets[i].widget_id not in fbl.client_manager.clients[fbl.widget_manager.widgets[i].client_id]['widgets']:
        fbl.client_manager.clients[fbl.widget_manager.widgets[i].client_id]['widgets'].append(fbl.widget_manager.widgets[i].widget_id)

# Load the diagram
my_client.tryComms({'widget':'GFX', 
                    'messageType': 'loadCircuitFromString', 
                    'data': {'string':data, 'name':'custom'}})

# Run the submodule called onGraphVizLoad2 to make the diagram interactive
my_client.tryComms({'widget':'GFX', 
                    'messageType': 'eval', 
                    'data': {'data':"window.fbl.loadSubmodule('data/FBLSubmodules/onGraphVizLoad2.js');", 'name':'custom'}})


In [164]:

# specify the pattern to replace and the replacement text
#this should match any R or L suffix and remove all text within ()

patterns = ['_[RL]','_\d','_C\d']
replacement_texts = ['','','']

# open the file in read mode
with open('G_auto.gv', 'r') as file:
    # read the file lines
    lines = file.readlines()
    oglines = lines
    
# replace the patterns in each line
for i in range(len(patterns)):
    lines = [re.sub(patterns[i], replacement_texts[i], line) for line in lines]


# open the file in write mode
with open(noiFileName + '.gv', 'w') as postProcessfile:
    # write the new lines to the file
    postProcessfile.writelines(lines)
    

# open the file containing the graph in read mode
with open(noiFileName + '_all.gv', 'w') as postProcessfile:
    # write the new lines to the file
    postProcessfile.writelines(oglines)